<a href="https://colab.research.google.com/github/ghasemieh/Duplicate-Bug-Identification-System/blob/master/Model_1_V04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Model-1-V4
# - Load data set
# - Remove NaN in short_desc
# - Convert to lowercase
# - Split the words using 1. ASCII character identification for english 2. Split by Space  3. wordninja
# - Apply normlise
# - Apply contractions/expansions 
# - Remove punctuations
# - remove tags.
# - remove special characters and digits.
# - Stemming/Lemmatisation. 
# - Prepare the whole data set
# - Calculate the similarity between every two bugreport for the same product

In [0]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [0]:
data = pd.read_csv('https://zenodo.org/record/400614/files/apache.csv?download=1',sep=',')

In [0]:
#Problem in reading csv file, cut the short_desc after 88th character
# print(data.iloc[[124]]['short_desc'])

In [0]:
# data.head()

In [0]:
data = data[['id','product','short_desc','dup_list']]
data.head()

,id,product,short_desc,dup_list
0,2,Log4j -,Just testing the Boogzeela setup for log4j,NaN
1,3,Log4j -,file sharing,NaN
2,22,Apache h,Need hooks for user-defined error handling,NaN
3,27,Log4j -,Error in the formatting of the (%F:%L) layout ...,NaN
4,29,Log4j -,"Calling Category.error(Object, Throwable) thro...",NaN


In [0]:
# Remove Blank Records
def remove_nan(df):
    print('Before Removing the NA:') 
    print(df.isnull().sum())
    blanks = []

    for index,id,product,short_desc,dup_list in data.itertuples():
        if type(short_desc)==str:
            if short_desc.isspace():
                blanks.append(index)
        else:
            blanks.append(index) 
    data.drop(blanks,inplace=True)
    
    return data.isnull().sum()

In [0]:
remove_nan(data)

Before Removing the NA:
id                0
product           0
short_desc       58
dup_list      41531
dtype: int64


id                0
product           0
short_desc        0
dup_list      41474
dtype: int64

In [0]:
# conda install -c conda-forge spacy
# !pip install wordninja
# !pip install normalise
# !pip install pycontractions
# conda install -c conda-forge spacy-lookups-data
# !python -m spacy download en_core_web_lg

In [0]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')
import nltk
# nltk.download('brown')
# nltk.download('names')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')
import wordninja as nj #for spliting the words in each documents
import normalise as ns 
from pycontractions import Contractions # for expansion and contrations
import contractions
import re # remove tags.

def preprocessing(df,id):
    # Extract and convert short_desc to string
    row = df[df['id'] == int(id)]                                # Extract a tuple from the dataframe
    short_desc_to_string = row.short_desc.to_string(index=False) # Conver short_desc to string
    short_desc_to_string = short_desc_to_string[1:]              # Remove the first space char from the begnning

#     my_short_desc_to_string = "I'd liked to said <pclass='test'> CANADAisn't Alireza@gmail.com the worst-countries in the 44 world!!!"
#     print("Original:\n" + my_short_desc_to_string)
    
    # Convert to lowercase
    short_desc_lowercase = short_desc_to_string.lower()
#     print("\nshort_desc_lowercase:\n" + short_desc_lowercase)
    
    # Split the words using 1. ASCII character identification for english 2. Split by Space  3. wordninja
    short_desc_splited = nj.split(short_desc_lowercase)
#     print("\nshort_desc_splited:\n" + str(short_desc_splited))
    
    # Apply normlise
    short_desc_normalised = ns.normalise(short_desc_splited,verbose=False)
    short_desc_normalised_listToStr = ' '.join(map(str, short_desc_normalised))
    
#     print("\nshort_desc_normalised:\n" + short_desc_normalised_listToStr)
    
    # Apply contractions/expansions 
#     cont = Contractions(api_key="glove-twitter-100") #We shold use this one but it shows error 
    short_desc_contract = contractions.fix(short_desc_normalised_listToStr) 
#     print("\nshort_desc_contract:\n" + str(short_desc_contract))
        
    # remove tags.
    clean = re.compile('<.*?>')
    short_desc_removed_tag = re.sub(clean, '', str(short_desc_contract)) 
#     print("\nshort_desc_removed_tag:\n" + str(short_desc_removed_tag))
    
    # remove special characters and digits.
    short_desc_removed_special_char = [re.sub(r"[^a-zA-Z]+", ' ', k) for k in str(short_desc_removed_tag).split("\n")]
#     print("\nshort_desc_removed_special_char:\n" + str(short_desc_removed_special_char))

    # Remove Punctuations and Stop words
    short_desc_doc = nlp(str(short_desc_removed_special_char))  
    short_desc_map = map(lambda token: token if (token.is_punct == False and token.is_stop == False) else None, short_desc_doc)
    short_desc_list = list(short_desc_map)        
    
    # Convert list to string and remove one-character word
    short_desc_string = ""     
    for element in short_desc_list:
        if element != None and len(element)>1:
            short_desc_string += str(element) +' '
    short_desc_string = short_desc_string[:-1]
#     print("\nshort_desc_string-1:\n" + short_desc_string)

#     short_desc_string2 = ' '.join(map(lambda x: str(x) if (x != None and len(x)>1) else None, short_desc_list))
#     print("\nshort_desc_string-2:\n" + short_desc_string2)
            
    # Stemming/Lemmatisation.       
    short_desc_lemmata = [token.lemma_ for token in nlp(short_desc_string)]
#     print("short_desc_lemmata:\n" + str(short_desc_lemmata))
    short_desc_preprocessed = ' '.join(map(str, short_desc_lemmata))
        
    return short_desc_preprocessed

In [0]:
# Prepare the vectors for each document
import time
import progressbar as pb

start_time = time.time()
#sample_size = 1000 # Sample Size
dataset_length = len(data)
#progress = pb.ProgressBar(maxval = sample_size).start()
progress = pb.ProgressBar(maxval = dataset_length).start()
progvar = 0

processed_data_list = []

for index,id,product,short_desc,dup_list in data.itertuples():
    string = preprocessing(data,id)
    processed_data_list.append((id,product,nlp(string)))   
    
    # Show the progress in the output
    progress.update(progvar+1)  
    progvar += 1    
    
    # Terminate the process when reach to sample size
#     if progvar >= sample_size:
#         break
# Convert list to dataframe
processed_data_df = pd.DataFrame(processed_data_list, columns=['id','product','short_desc'])
# show the time of process        
print("--- %s seconds ---" % (time.time() - start_time))  

 86% (37840 of 43991) |###############   | Elapsed Time: 1:02:23 ETA:   1:09:55

In [0]:
remove_nan(processed_data_df)

In [0]:
# Calculate one by one similarity score
start_time = time.time()
dataset_length = len(processed_data_df)
progress = pb.ProgressBar(maxval = dataset_length).start()
progvar = 0

similarities_score_list = []

for index1,id1,product1,short_desc1 in processed_data_df.itertuples():
    for index2,id2,product2,short_desc2 in processed_data_df.itertuples():
        if (id1 != id2) and (product1 == product2): # if two bug reports belong to the same product then check the similarity
            similarity_score = short_desc1.similarity(short_desc2)
            similarities_score_list.append((id1,id2,product1,similarity_score))
            
    # Show the progress in the output        
    progress.update(progvar+1)  
    progvar += 1 

# Sort based on the score and conver to dataframe
sorted_similarities_score = sorted(similarities_score_list, key=lambda tup: tup[3], reverse=True)
similarities_score_df = pd.DataFrame(sorted_similarities_score, columns=['id1','id2','product','score'] )

print("--- %s seconds ---" % (time.time() - start_time)) # show the time of process

In [0]:
similarities_score_df.iloc[:10]

**Assessment of the accuracy**